In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy
from scipy.special import rel_entr

**Data Proprocessing**

In [2]:
# Load raw data
df_plots = pd.read_csv("/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/Field/01_plot_identification.csv")
df_trees = pd.read_csv("/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/Field/03_tree.csv")
FF_treelist = pd.read_csv("/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/FF_treelist_all.csv")
pft_mapping = pd.read_csv("/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/FIATreeSpeciesCode_pft.csv", delimiter=";")

# --- Clean inconsistencies in scientific names in field data ---
df_trees['tree_sp_scientific_name'] = df_trees['tree_sp_scientific_name'].replace({
    'Salix sp.': 'Salix spp.',
    'Quercus sp.': 'Quercus spp.',
    'Cornus nuttalii': 'Cornus nuttallii'
})

# --- Clean and standardize PFT mapping ---
pft_mapping.rename(columns={"PFT,": "PFT"}, inplace=True)                         # Fix malformed column name
pft_mapping["PFT"] = pft_mapping["PFT"].str.strip(',')                            # Remove trailing commas
pft_mapping["PFT"] = pft_mapping["PFT"].replace({'Deciduous': 'Deciduous Broadleaf'})  # Normalize naming

# --- Normalize site name in plots and merge with tree-level data ---
df_plots['site_name'] = df_plots['site_name'].str.lower()
df_trees = df_trees.merge(df_plots[["site_name", "inventory_id"]], on="inventory_id", how="left")

# --- Map field trees to PFT and drop invalid/missing entries ---
field_labeled = df_trees.merge(pft_mapping[['SCI_NAME', 'PFT']], left_on='tree_sp_scientific_name', right_on='SCI_NAME', how='left')
field_labeled = field_labeled.dropna(subset=['tree_ht', 'tree_dbh', 'PFT']).copy()

# --- Rename and extract common columns from field data ---
field_labeled = field_labeled.rename(columns={'tree_ht': 'HT', 'tree_dbh': 'DIA'})
field_labeled['GENUS'] = field_labeled['tree_sp_scientific_name'].str.split().str[0]
field_labeled['species_name'] = field_labeled['tree_sp_scientific_name']
field_final = field_labeled[['HT', 'DIA', 'PFT', 'GENUS', 'species_name', 'site_name', 'inventory_id']].copy()
field_final['source'] = 'FIELD'  # Add source label

# --- Process FastFuel tree list: join with PFT, drop invalids ---
ff_labeled = FF_treelist.merge(pft_mapping[['SPCD', 'PFT', 'SCI_NAME']], on='SPCD', how='left')
ff_labeled = ff_labeled.dropna(subset=['HT', 'DIA', 'PFT']).copy()

# --- Extract and normalize common fields from FF data ---
ff_labeled = ff_labeled.rename(columns={'geometry': 'inventory_id'})
ff_labeled['GENUS'] = ff_labeled['SCI_NAME'].str.split().str[0]
ff_labeled['species_name'] = ff_labeled['SCI_NAME']
ff_labeled['site_name'] = ff_labeled['site_name'].str.lower()
ff_final = ff_labeled[['HT', 'DIA', 'PFT', 'GENUS', 'species_name', 'site_name', 'inventory_id']].copy()
ff_final['source'] = 'FF'  # Add source label

# --- Combine field and FastFuel data into unified dataset ---
combined_df = pd.concat([field_final, ff_final], ignore_index=True)

In [3]:
# --- Normalize and preprocess ---
combined_df['site_name'] = combined_df['site_name'].str.lower()
combined_df['PFT'] = combined_df['PFT'].str.title().str.strip()
combined_df['GENUS'] = combined_df['GENUS'].str.title().str.strip()
combined_df['species_name'] = combined_df['species_name'].str.title().str.strip()
combined_df = combined_df.dropna(subset=['HT', 'DIA', 'PFT', 'GENUS', 'species_name', 'site_name'])

# --- Label encoders ---
pft_encoder = LabelEncoder()
genus_encoder = LabelEncoder()
species_encoder = LabelEncoder()
site_encoder = LabelEncoder()

combined_df['site_code'] = site_encoder.fit_transform(combined_df['site_name'])

# --- PFT MODEL ---
pft_df = combined_df[['HT', 'DIA', 'site_code', 'PFT']].copy()
pft_df['PFT_encoded'] = pft_encoder.fit_transform(pft_df['PFT'])

X_pft = pft_df[['HT', 'DIA', 'site_code']]
y_pft = pft_df['PFT_encoded']

pft_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    random_state=42,
    n_jobs=-1
).fit(X_pft, y_pft)

combined_df['PFT_encoded'] = pft_encoder.transform(combined_df['PFT'])

# --- GENUS MODEL ---
genus_df = combined_df[['HT', 'DIA', 'PFT_encoded', 'site_code', 'GENUS']].copy()
genus_df['GENUS_encoded'] = genus_encoder.fit_transform(genus_df['GENUS'])

X_genus = genus_df[['HT', 'DIA', 'PFT_encoded', 'site_code']]
y_genus = genus_df['GENUS_encoded']

genus_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
).fit(X_genus, y_genus)

combined_df['GENUS_encoded'] = genus_encoder.transform(combined_df['GENUS'])

# --- SPECIES MODEL ---
species_df = combined_df[['HT', 'DIA', 'PFT_encoded', 'GENUS_encoded', 'site_code', 'species_name']].copy()
species_df['species_encoded'] = species_encoder.fit_transform(species_df['species_name'])

X_species = species_df[['HT', 'DIA', 'PFT_encoded', 'GENUS_encoded', 'site_code']]
y_species = species_df['species_encoded']

species_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
).fit(X_species, y_species)

In [4]:
def preprocess_tls_fixed(tls_csv_path, plot_lookup_csv_path):
    """
    Loads and preprocesses TLS-derived tree list data.

    This function performs the following steps:
    - Loads TLS tree data and a plot-to-site lookup table.
    - Merges site information into the TLS data using 'plot_blk'.
    - Renames 'H' and 'DBH' columns to 'HT' and 'DIA' for consistency.
    - Drops rows with missing height, diameter, or site information.

    Parameters:
        tls_csv_path (str): Path to the TLS tree list CSV file.
        plot_lookup_csv_path (str): Path to the plot-to-site lookup CSV file.

    Returns:
        pd.DataFrame: A cleaned and standardized TLS tree list DataFrame.
    """
    tls_df = pd.read_csv(tls_csv_path)
    plot_lookup = pd.read_csv(plot_lookup_csv_path)
    plot_lookup['site_name'] = plot_lookup['site_name'].str.lower()
    tls_df = tls_df.merge(plot_lookup[['plot_blk', 'site_name']], on='plot_blk', how='left')
    tls_df = tls_df.rename(columns={'H': 'HT', 'DBH': 'DIA'})
    tls_df = tls_df.dropna(subset=['HT', 'DIA', 'site_name'])

    return tls_df

Predict TLS data and generate results saved in dataframe

In [5]:
# --- Preprocess TLS data ---
tls_df = preprocess_tls_fixed(
    tls_csv_path="/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/TLS/TLS_treelist.csv",
    plot_lookup_csv_path="/home/jovyan/work/SDHCN-Shared-Storage-214ea/sprint_4/data/TLS/blk_plot_identification.csv"
)

# Add site_code if not already there
if 'site_name' in tls_df.columns:
    tls_df['site_code'] = tls_df['site_name'].astype('category').cat.codes

# PFT Prediction
pft_features = tls_df[['HT', 'DIA', 'site_code']]
pft_preds_encoded = pft_model.predict(pft_features)
pft_preds = pft_encoder.inverse_transform(pft_preds_encoded)
tls_df['PFT'] = pft_preds
tls_df['PFT_encoded'] = pft_preds_encoded

# GENUS Prediction
genus_features = tls_df[['HT', 'DIA', 'PFT_encoded', 'site_code']]
genus_preds_encoded = genus_model.predict(genus_features)
genus_preds = genus_encoder.inverse_transform(genus_preds_encoded)
tls_df['GENUS'] = genus_preds
tls_df['GENUS_encoded'] = genus_preds_encoded

# SPECIES Prediction
species_features = tls_df[['HT', 'DIA', 'PFT_encoded', 'GENUS_encoded', 'site_code']]
species_preds_encoded = species_model.predict(species_features)
species_preds = species_encoder.inverse_transform(species_preds_encoded)
tls_df['species_name'] = species_preds

# --- Create and normalize distribution DataFrames ---
pft_dist = tls_df.groupby(['site_name', 'plot_blk', 'PFT']).size().unstack(fill_value=0)
pft_dist = pft_dist.div(pft_dist.sum(axis=1), axis=0)
pft_dist = pd.DataFrame(pft_dist).reset_index()

genus_dist = tls_df.groupby(['site_name', 'plot_blk', 'GENUS']).size().unstack(fill_value=0)
genus_dist = genus_dist.div(genus_dist.sum(axis=1), axis=0)
genus_dist = pd.DataFrame(genus_dist).reset_index()

species_dist = tls_df.groupby(['site_name','plot_blk', 'species_name']).size().unstack(fill_value=0)
species_dist = species_dist.div(species_dist.sum(axis=1), axis=0)
species_dist = pd.DataFrame(species_dist).reset_index()

Now you can access a site's **PFT/Genus/Species Distribution**.  
An example of accessing the **PFT distribution** of site `CATCU_0121_20241012_1` is shown below:

In [6]:
plot_n = "CATCU_0121_20241012_1" # change here for plot_blk
pft_site_dist = pft_dist[pft_dist['plot_blk'] == plot_n]
print(pft_site_dist)

PFT site_name               plot_blk  Deciduous Broadleaf  Evergreen  \
137       win  CATCU_0121_20241012_1             0.066667        0.0   

PFT  Evergreen Broadleaf  Evergreen Conifer  
137                  0.0           0.933333  


Next, we evaluate our model based on **KL Divergence**

In [7]:
# --- KL divergence Evaluation ---
def kl_divergence(p, q):
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    p = np.where(p == 0, 1e-10, p)
    q = np.where(q == 0, 1e-10, q)
    return np.sum(rel_entr(p, q))

def compare_distributions(tls_df, field_df, column, site_column='site_name'):
    results = []
    for site in sorted(set(tls_df[site_column].unique()).intersection(set(field_df[site_column].unique()))):
        tls_sub = tls_df[tls_df[site_column] == site]
        field_sub = field_df[field_df[site_column] == site]
        tls_dist = tls_sub[column].value_counts(normalize=True)
        field_dist = field_sub[column].value_counts(normalize=True)
        keys = sorted(set(tls_dist.index).union(field_dist.index))
        tls_vec = np.array([tls_dist.get(k, 0) for k in keys])
        field_vec = np.array([field_dist.get(k, 0) for k in keys])
        kl = kl_divergence(field_vec, tls_vec)
        results.append({'site': site, 'type': column, 'KL divergence': kl})
    return pd.DataFrame(results)

# --- Generate comparison report ---
report_parts = []
for col in ['PFT', 'GENUS', 'species_name']:
    report_parts.append(compare_distributions(tls_df, field_labeled, col))

kl_report_df = pd.concat(report_parts, ignore_index=True)
print(kl_report_df)

   site          type  KL divergence
0   ind           PFT      21.657137
1   puc           PFT      22.081241
2   sdr           PFT      22.587276
3   sha           PFT      21.913482
4   tcu           PFT      22.795039
5   win           PFT      22.347565
6   ind         GENUS       1.633912
7   puc         GENUS       1.273829
8   sdr         GENUS       0.000000
9   sha         GENUS       2.915229
10  tcu         GENUS       2.324185
11  win         GENUS       5.088356
12  ind  species_name      21.643129
13  puc  species_name      21.883495
14  sdr  species_name      22.534280
15  sha  species_name      21.820378
16  tcu  species_name      21.824750
17  win  species_name      21.303896
